In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ============================================
# CORE PYTHON & DATA HANDLING
# ============================================
import numpy as np
import pandas as pd
import os
import re
from collections import Counter

# ============================================
# VISUALIZATION
# ============================================
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# ============================================
# NLP PREPROCESSING (NLTK)
# ============================================
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# ============================================
# CLASSICAL MACHINE LEARNING
# ============================================
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

# ============================================
# STATISTICAL TESTS
# ============================================
from scipy.stats import ttest_ind, mannwhitneyu, chi2_contingency, shapiro

# ============================================
# SPARSE MATRIX HANDLING
# ============================================
from scipy.sparse import hstack

# ============================================
# DEEP LEARNING (TF/KERAS)
# ============================================
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Embedding, LSTM, GRU, Bidirectional,
    Conv1D, GlobalMaxPooling1D, Dense, Dropout
)

# ============================================
# TRANSFORMERS (HUGGINGFACE)
# ============================================
from transformers import (
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification
)

# ============================================
# MODEL EXPLAINABILITY
# ============================================
import shap
from lime.lime_text import LimeTextExplainer

# ============================================
# UTILITY
# ============================================
import warnings
warnings.filterwarnings("ignore")


In [3]:
## 2. Load cleaned datasets

isot = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_clean/clean_isot.csv")
wel  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_clean/clean_welfake.csv")

# Merge datasets
df = pd.concat([isot, wel], ignore_index=True)

# Features and labels
X = df["text_clean"].astype(str)
y = df["label"].astype(int)

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=47,
    stratify=y
)

print("Merged shape:", df.shape)
print("Train size:", len(X_train_text))
print("Test size:", len(X_test_text))


Merged shape: (101131, 4)
Train size: 80904
Test size: 20227


In [4]:
# -------------------------
# Tokenizer
# -------------------------
VOCAB_SIZE = 50000
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)

# -------------------------
# Padding
# -------------------------
MAX_LEN = 300  # good length for news data

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_LEN, padding="post", truncating="post")

# -------------------------
# Compute Class Weights
# -------------------------
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))

print("Class Weights:", class_weights)


Class Weights: {0: np.float64(0.9726844282004424), 1: np.float64(1.0288940889205413)}


In [5]:
# ============================================================
# Build BiLSTM Model
# ============================================================

EMBED_DIM = 128
LSTM_UNITS = 128

model_bilstm = Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBED_DIM, input_length=MAX_LEN),

    # Bidirectional LSTM
    tf.keras.layers.Bidirectional(LSTM(LSTM_UNITS, return_sequences=False)),

    Dropout(0.3),
    Dense(64, activation="relu"),

    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

# Compile
model_bilstm.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_bilstm.summary()

# ============================================================
# Train the Model
# ============================================================

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

history_bilstm = model_bilstm.fit(
    X_train_pad,
    y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=128,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# ============================================================
# Evaluate on Test Set
# ============================================================

pred_probs = model_bilstm.predict(X_test_pad)
y_pred = (pred_probs > 0.5).astype(int)

print("\n=== BiLSTM Results ===")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
569/569 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - accuracy: 0.5644 - loss: 0.6541 - val_accuracy: 0.5797 - val_loss: 0.5997
Epoch 2/5
569/569 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.6011 - loss: 0.5811 - val_accuracy: 0.5983 - val_loss: 0.5962
Epoch 3/5
569/569 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - accuracy: 0.6084 - loss: 0.5672 - val_accuracy: 0.5798 - val_loss: 0.5982
Epoch 4/5
569/569 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - accuracy: 0.6099 - loss: 0.5601 - val_accuracy: 0.5832 - val_loss: 0.6439
633/633 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step

=== BiLSTM Results ===
              precision    recall  f1-score   support

           0       0.58      0.70      0.64     10397
           1       0.60      0.47      0.53      9830

    accuracy                           0.59     20227
   macro avg       0.59      0.59      0.58     20227
weighted avg       0.59      0.59      0.58     20227

Accuracy: 0.5892618776882385


In [6]:
# =======================================================
#  CNN + LSTM Hybrid Model
# =======================================================

def build_cnn_lstm(vocab_size, max_len):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),

        # 1D CNN layer
        Conv1D(filters=128, kernel_size=5, activation='relu'),

        # Max pooling to reduce sequence size
        GlobalMaxPooling1D(),

        # LSTM block
        tf.keras.layers.Reshape((1, 128)),  # reshaping for LSTM
        LSTM(64, return_sequences=False),

        # Dense layers
        Dense(64, activation='relu'),
        Dropout(0.3),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        loss="binary_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )

    return model

cnn_lstm_model = build_cnn_lstm(VOCAB_SIZE, MAX_LEN)
cnn_lstm_model.summary()

history_cnn_lstm = cnn_lstm_model.fit(
    X_train_pad, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    ]
)

# Evaluation
y_pred_probs = cnn_lstm_model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n=== CNN-LSTM Results ===")
print(classification_report(y_test, y_pred))

acc_cnn_lstm = accuracy_score(y_test, y_pred)
print("Accuracy:", acc_cnn_lstm)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
506/506 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5401 - loss: 0.6656 - val_accuracy: 0.5929 - val_loss: 0.5824
Epoch 2/3
506/506 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6063 - loss: 0.5658 - val_accuracy: 0.5908 - val_loss: 0.5874
Epoch 3/3
506/506 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.6185 - loss: 0.5391 - val_accuracy: 0.5931 - val_loss: 0.6078
633/633 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

=== CNN-LSTM Results ===
              precision    recall  f1-score   support

           0       0.93      0.24      0.38     10397
           1       0.55      0.98      0.70      9830

    accuracy                           0.60     20227
   macro avg       0.74      0.61      0.54     20227
weighted avg       0.75      0.60      0.54     20227

Accuracy: 0.5984575072922331
